In [318]:
import pandas as pd

first_buildings = pd.read_csv("sql/first_buildings.csv")
print(first_buildings)

       gametime         platformgameid  teamid buildingtype
0        710694  ESPORTSTMNT01:1431635     100       turret
1       1863543  ESPORTSTMNT03:2073826     100    inhibitor
2        655449  ESPORTSTMNT02:1331126     200       turret
3       1458069  ESPORTSTMNT03:2542608     200    inhibitor
4       1018478  ESPORTSTMNT05:2942358     100       turret
...         ...                    ...     ...          ...
74992   1440023  ESPORTSTMNT05:1870222     100    inhibitor
74993   2349105  ESPORTSTMNT01:2693353     200    inhibitor
74994    598202  ESPORTSTMNT06:1720693     200       turret
74995    988622  ESPORTSTMNT01:3408793     200       turret
74996   1606464  ESPORTSTMNT03:3185926     100    inhibitor

[74997 rows x 4 columns]


In [319]:
first_inhib = first_buildings[first_buildings["buildingtype"] == "inhibitor"]
first_inhib = first_inhib.sort_values(by=['platformgameid', 'gametime'])
first_inhib["killerteamid_inhib"] = first_inhib["teamid"].apply(lambda x: 200 if x == 100 else 100)
first_inhib.drop_duplicates(subset=['platformgameid'], keep='first', inplace=True)
first_inhib.drop(columns=["buildingtype", "teamid"], inplace=True)
first_inhib.rename(columns={"gametime": "gametime_inhib"}, inplace=True)
display(first_inhib)

,gametime_inhib,platformgameid,killerteamid_inhib
70824,1708748,ESPORTSTMNT01:1110148,100
41143,1602302,ESPORTSTMNT01:1110162,100
62898,1546286,ESPORTSTMNT01:1110177,100
3623,1628728,ESPORTSTMNT01:1110187,100
50722,1694081,ESPORTSTMNT01:1110198,100
...,...,...,...
2262,1700113,LPL_B:330027,100
35688,1658796,LPL_B:330032,200
11354,1946801,LPL_B:330033,200
52312,1447915,LPL_B:330039,200


In [320]:
#first tower
first_tower = first_buildings[first_buildings["buildingtype"] == "turret"]
first_tower = first_tower.sort_values(by=['platformgameid', 'gametime'])
first_tower["killerteamid_tower"] = first_tower["teamid"].apply(lambda x: 200 if x == 100 else 100)
first_tower.drop_duplicates(subset=['platformgameid'], keep='first', inplace=True)
first_tower.drop(columns=["buildingtype", "teamid"], inplace=True)
first_tower.rename(columns={"gametime": "gametime_tower"}, inplace=True)

In [321]:
buildings_joined = first_inhib.merge(first_tower, on="platformgameid")

In [322]:
first_kill = pd.read_csv("sql/first_blood.csv")
first_kill = first_kill.sort_values(by=['platformgameid', 'gametime'])
first_kill.drop_duplicates(subset=['platformgameid'], keep='first', inplace=True)
display(first_kill)
first_kill.rename(columns={
    "gametime": "gametime_kill",
    "killerteamid": "killerteamid_kill",
}, inplace=True)

,gametime,platformgameid,killerteamid
4535,245612,ESPORTSTMNT01:1110148,100
9142,192537,ESPORTSTMNT01:1110162,100
21281,207700,ESPORTSTMNT01:1110177,100
35484,359245,ESPORTSTMNT01:1110187,200
11323,148483,ESPORTSTMNT01:1110198,100
...,...,...,...
21378,294277,LPL_B:330027,100
13430,227140,LPL_B:330032,100
44132,505054,LPL_B:330033,100
17859,302191,LPL_B:330039,100


In [323]:
# add start times
df = buildings_joined.merge(first_kill, on="platformgameid")
event_start_times = pd.read_csv("sql/event_start_times.csv")
df = df.merge(event_start_times, on="platformgameid")

In [324]:
# add team games 
team_games = pd.read_csv("sql/team_games.csv")
team_games.sort_values(by=['platformgameid'], inplace=True)

with_teams = df.merge(team_games, on="platformgameid")

blue_side = with_teams[with_teams["side"] == 100]
blue_side.rename(columns={
    "teamid": "blue_teamid",
}, inplace=True)
blue_side.drop(columns=["side"], inplace=True)
red_side = with_teams[with_teams["side"] == 200]
red_side.rename(columns={
    "teamid": "red_teamid",
}, inplace=True)
red_side.drop(columns=[
    "side",
    "esportsgameid",
    "killerteamid_inhib",
    "killerteamid_tower",
    "killerteamid_kill",
    "gametime_inhib",
    "gametime_tower",
    "gametime_kill",
    "eventtime",
], inplace=True)

joined_side = blue_side.merge(red_side, on="platformgameid")

display(joined_side)
# joined_side.to_csv("joined.csv", index=False)

C:\Users\Jan\AppData\Local\Temp\ipykernel_28060\3386314402.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blue_side.rename(columns={
C:\Users\Jan\AppData\Local\Temp\ipykernel_28060\3386314402.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blue_side.drop(columns=["side"], inplace=True)
C:\Users\Jan\AppData\Local\Temp\ipykernel_28060\3386314402.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  red_side.rename(columns={
C:\U

,gametime_inhib,platformgameid,killerteamid_inhib,gametime_tower,killerteamid_tower,gametime_kill,killerteamid_kill,eventtime,blue_teamid,esportsgameid,red_teamid
0,1708748,ESPORTSTMNT01:1110148,100,939441,100,245612,100,2019-05-01T09:11:39.980Z,1.007832e+17,101978171958054800,9.956641e+16
1,1602302,ESPORTSTMNT01:1110162,100,677086,100,192537,100,2019-05-01T10:15:29.004Z,9.876799e+16,101978171963953057,9.956641e+16
2,1546286,ESPORTSTMNT01:1110177,100,836608,100,207700,100,2019-05-01T11:15:13.609Z,1.007832e+17,101978171979157453,9.956641e+16
3,1628728,ESPORTSTMNT01:1110187,100,626524,200,359245,200,2019-05-01T12:09:47.993Z,9.876799e+16,101978171965722534,9.956641e+16
4,1694081,ESPORTSTMNT01:1110198,100,895424,100,148483,100,2019-05-01T13:01:30.206Z,9.956641e+16,101978171973586877,9.956641e+16
...,...,...,...,...,...,...,...,...,...,...,...
25182,1700113,LPL_B:330027,100,885600,100,294277,100,2023-07-17T09:14:18.420Z,9.876799e+16,110428848767940944,9.956640e+16
25183,1658796,LPL_B:330032,200,687897,200,227140,100,2023-07-17T10:03:31.166Z,9.956640e+16,110428848767940945,9.876799e+16
25184,1946801,LPL_B:330033,200,900364,200,505054,100,2023-07-17T11:17:17.592Z,9.956640e+16,110428848767875248,9.876799e+16
25185,1447915,LPL_B:330039,200,1008863,100,302191,100,2023-07-17T12:16:05.231Z,9.876799e+16,110428848767875249,9.956640e+16


In [325]:
shutdown = pd.read_csv('sql/shutdown.csv')
shutdown.sort_values(by=["platformgameid", "gametime"], inplace=True)
shutdown.head()

,platformgameid,eventtime,gametime,shutdown_held_1,shutdown_held_2,shutdown_held_3,shutdown_held_4,shutdown_held_5,shutdown_held_6,shutdown_held_7,shutdown_held_8,shutdown_held_9,shutdown_held_10
326292,ESPORTSTMNT01:1310842,2020-03-13T05:30:17.724Z,705490,50.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65529,ESPORTSTMNT01:1310842,2020-03-13T05:31:46.811Z,794576,50.0,0.0,100.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0
65530,ESPORTSTMNT01:1310842,2020-03-13T05:32:05.244Z,813008,50.0,0.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36108,ESPORTSTMNT01:1310842,2020-03-13T05:33:24.093Z,891854,50.0,0.0,200.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0
208900,ESPORTSTMNT01:1310842,2020-03-13T05:33:32.285Z,900047,50.0,0.0,200.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0


In [326]:
new_row = {
    "platformgameid": None,
    "gametime": None,
    "shutdown_held_1": None,
    "shutdown_held_2": None,
    "shutdown_held_3": None,
    "shutdown_held_4": None,
    "shutdown_held_5": None,
    "shutdown_held_6": None,
    "shutdown_held_7": None,
    "shutdown_held_8": None,
    "shutdown_held_9": None,
    "shutdown_held_10": None,
}

new = []
prev_row = None
for _, row in shutdown.iterrows():
    if prev_row is None:
        prev_row = row
        new_row["platformgameid"] = row["platformgameid"]
        new_row["gametime"] = row["gametime"]
        for i in range(1, 11):
            new_row["shutdown_held_" + str(i)] = row["shutdown_held_" + str(i)]
        continue
    elif row["platformgameid"] != prev_row["platformgameid"]:
        new.append(new_row)
        new_row = {
            "platformgameid": row["platformgameid"],
            "gametime": row["gametime"],
        }
        for i in range(1, 11):
            new_row["shutdown_held_" + str(i)] = row["shutdown_held_" + str(i)]
    elif row["platformgameid"] == new_row["platformgameid"]:
        for i in range(1, 11):
            if row["shutdown_held_" + str(i)] > prev_row["shutdown_held_" + str(i)]:
                new_row["shutdown_held_" + str(i)] += row["shutdown_held_" + str(i)]- prev_row["shutdown_held_" + str(i)] 


    prev_row = row

df = pd.DataFrame(new)
display(df)

,platformgameid,gametime,shutdown_held_1,shutdown_held_2,shutdown_held_3,shutdown_held_4,shutdown_held_5,shutdown_held_6,shutdown_held_7,shutdown_held_8,shutdown_held_9,shutdown_held_10
0,ESPORTSTMNT01:1310842,705490,500.0,150.0,650.0,850.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ESPORTSTMNT01:1310866,1056752,600.0,0.0,1550.0,600.0,0.0,0.0,150.0,400.0,700.0,0.0
2,ESPORTSTMNT01:1310892,917488,250.0,0.0,450.0,0.0,200.0,0.0,0.0,150.0,100.0,0.0
3,ESPORTSTMNT01:1313641,1367952,100.0,0.0,0.0,550.0,0.0,300.0,0.0,100.0,850.0,0.0
4,ESPORTSTMNT01:1313662,1574820,300.0,500.0,650.0,650.0,0.0,50.0,0.0,50.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16895,LPL_B:329904,351030,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150.0,0.0
16896,LPL_B:329939,725032,300.0,500.0,750.0,750.0,0.0,0.0,0.0,0.0,0.0,150.0
16897,LPL_B:329985,547021,250.0,200.0,250.0,550.0,0.0,0.0,0.0,150.0,50.0,0.0
16898,LPL_B:330027,746019,450.0,150.0,350.0,400.0,0.0,50.0,0.0,0.0,250.0,0.0


In [327]:
shutdown = df
shutdown["blue_team_shutdown"] =  df["shutdown_held_1"] + df["shutdown_held_2"] + df["shutdown_held_3"] + df["shutdown_held_4"] + df["shutdown_held_5"]
shutdown["red_team_shutdown"] = df["shutdown_held_6"] + df["shutdown_held_7"] + df["shutdown_held_8"] + df["shutdown_held_9"] + df["shutdown_held_10"]
shutdown.fillna(0, inplace=True)
shutdown = shutdown.drop(columns=[
    "shutdown_held_1",
    "shutdown_held_2",
    "shutdown_held_3",
    "shutdown_held_4",
    "shutdown_held_5",
    "shutdown_held_6",
    "shutdown_held_7",
    "shutdown_held_8",
    "shutdown_held_9",
    "shutdown_held_10",
])
display(shutdown)

,platformgameid,gametime,blue_team_shutdown,red_team_shutdown
0,ESPORTSTMNT01:1310842,705490,2150.0,0.0
1,ESPORTSTMNT01:1310866,1056752,2750.0,1250.0
2,ESPORTSTMNT01:1310892,917488,900.0,250.0
3,ESPORTSTMNT01:1313641,1367952,650.0,1250.0
4,ESPORTSTMNT01:1313662,1574820,2100.0,100.0
...,...,...,...,...
16895,LPL_B:329904,351030,50.0,150.0
16896,LPL_B:329939,725032,2300.0,150.0
16897,LPL_B:329985,547021,1250.0,200.0
16898,LPL_B:330027,746019,1350.0,300.0


In [328]:
shutdown_collected = pd.read_csv('sql/shutdown_collected.csv')
shutdown_collected.fillna(0, inplace=True)
shutdown_collected.sort_values(by=["platformgameid"], inplace=True)

blue_side = shutdown_collected[shutdown_collected["side"] == 100]
blue_side = blue_side.rename(columns={
    "shutdownbounty": "blue_shutdown_collected",
})
blue_side.drop(columns=["side", "teamid"], inplace=True)
red_side = shutdown_collected[shutdown_collected["side"] == 200]
red_side = red_side.rename(columns={
    "shutdownbounty": "red_shutdown_collected",
})
red_side.drop(columns=[
    "side",
    "teamid",
], inplace=True)

shutdown_collected = blue_side.merge(red_side, on="platformgameid")
display(shutdown_collected)


,platformgameid,blue_shutdown_collected,red_shutdown_collected
0,ESPORTSTMNT01:1110148,0.0,0.0
1,ESPORTSTMNT01:1110162,0.0,0.0
2,ESPORTSTMNT01:1110177,0.0,0.0
3,ESPORTSTMNT01:1110187,0.0,0.0
4,ESPORTSTMNT01:1110198,0.0,0.0
...,...,...,...
25010,LPL_B:330027,400.0,500.0
25011,LPL_B:330032,100.0,800.0
25012,LPL_B:330033,1300.0,1600.0
25013,LPL_B:330039,0.0,0.0


In [329]:
df = shutdown.merge(shutdown_collected, on="platformgameid")
display(df)

,platformgameid,gametime,blue_team_shutdown,red_team_shutdown,blue_shutdown_collected,red_shutdown_collected
0,ESPORTSTMNT01:1310842,705490,2150.0,0.0,0.0,0.0
1,ESPORTSTMNT01:1310866,1056752,2750.0,1250.0,0.0,0.0
2,ESPORTSTMNT01:1310892,917488,900.0,250.0,0.0,0.0
3,ESPORTSTMNT01:1313641,1367952,650.0,1250.0,0.0,0.0
4,ESPORTSTMNT01:1313662,1574820,2100.0,100.0,0.0,0.0
...,...,...,...,...,...,...
16777,LPL_B:329904,351030,50.0,150.0,1000.0,150.0
16778,LPL_B:329939,725032,2300.0,150.0,100.0,200.0
16779,LPL_B:329985,547021,1250.0,200.0,400.0,800.0
16780,LPL_B:330027,746019,1350.0,300.0,400.0,500.0


In [330]:
hmm = df[(df["blue_team_shutdown"] < df["red_shutdown_collected"]) | (df["red_team_shutdown"] < df["blue_shutdown_collected"])]
display(hmm)

,platformgameid,gametime,blue_team_shutdown,red_team_shutdown,blue_shutdown_collected,red_shutdown_collected
1345,ESPORTSTMNT01:2730855,1352027,50.0,600.0,0.0,250.0
1346,ESPORTSTMNT01:2730866,1379018,1350.0,0.0,150.0,850.0
1347,ESPORTSTMNT01:2731026,341021,1000.0,1400.0,1550.0,400.0
1349,ESPORTSTMNT01:2731088,829026,150.0,350.0,800.0,150.0
1354,ESPORTSTMNT01:2731711,831031,1900.0,0.0,300.0,700.0
...,...,...,...,...,...,...
16775,LPL_B:329728,700000,2100.0,300.0,700.0,1900.0
16777,LPL_B:329904,351030,50.0,150.0,1000.0,150.0
16779,LPL_B:329985,547021,1250.0,200.0,400.0,800.0
16780,LPL_B:330027,746019,1350.0,300.0,400.0,500.0


In [331]:

df["blue_shutdown_converted"] = df["red_team_shutdown"] - (df["red_team_shutdown"] - df["blue_shutdown_collected"])
df["red_shutdown_converted"] = df["blue_team_shutdown"] - (df["blue_team_shutdown"] - df["red_shutdown_collected"])
display(df)

,platformgameid,gametime,blue_team_shutdown,red_team_shutdown,blue_shutdown_collected,red_shutdown_collected,blue_shutdown_converted,red_shutdown_converted
0,ESPORTSTMNT01:1310842,705490,2150.0,0.0,0.0,0.0,0.0,0.0
1,ESPORTSTMNT01:1310866,1056752,2750.0,1250.0,0.0,0.0,0.0,0.0
2,ESPORTSTMNT01:1310892,917488,900.0,250.0,0.0,0.0,0.0,0.0
3,ESPORTSTMNT01:1313641,1367952,650.0,1250.0,0.0,0.0,0.0,0.0
4,ESPORTSTMNT01:1313662,1574820,2100.0,100.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
16777,LPL_B:329904,351030,50.0,150.0,1000.0,150.0,1000.0,150.0
16778,LPL_B:329939,725032,2300.0,150.0,100.0,200.0,100.0,200.0
16779,LPL_B:329985,547021,1250.0,200.0,400.0,800.0,400.0,800.0
16780,LPL_B:330027,746019,1350.0,300.0,400.0,500.0,400.0,500.0


In [332]:
final = joined_side.merge(df, on="platformgameid")
# joined_side = joined_side.merge(shutdown_collected, on="platformgameid")
display(final)
final.to_csv("joined.csv", index=False)

,gametime_inhib,platformgameid,killerteamid_inhib,gametime_tower,killerteamid_tower,gametime_kill,killerteamid_kill,eventtime,blue_teamid,esportsgameid,red_teamid,gametime,blue_team_shutdown,red_team_shutdown,blue_shutdown_collected,red_shutdown_collected,blue_shutdown_converted,red_shutdown_converted
0,1591696,ESPORTSTMNT01:1310842,100,755269,100,274459,100,2020-03-13T05:16:50.558Z,9.876799e+16,103535402770536790,9.956641e+16,705490,2150.0,0.0,0.0,0.0,0.0,0.0
1,2520557,ESPORTSTMNT01:1310866,100,840724,200,423171,200,2020-03-13T07:17:46.390Z,9.956641e+16,103535402770536772,1.013838e+17,1056752,2750.0,1250.0,0.0,0.0,0.0,0.0
2,1982488,ESPORTSTMNT01:1310892,200,608609,100,568211,100,2020-03-13T08:29:27.072Z,9.956641e+16,103535402770536804,9.956641e+16,917488,900.0,250.0,0.0,0.0,0.0,0.0
3,2561230,ESPORTSTMNT01:1313641,200,962342,200,381466,200,2020-03-20T19:08:34.415Z,9.956641e+16,103462454882440906,9.956641e+16,1367952,650.0,1250.0,0.0,0.0,0.0,0.0
4,1890504,ESPORTSTMNT01:1313662,100,978916,200,218111,100,2020-03-20T20:25:03.986Z,1.034620e+17,103462454882440842,1.034620e+17,1574820,2100.0,100.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16774,1716372,LPL_B:329904,200,854089,100,81948,100,2023-07-16T09:19:18.628Z,1.013889e+17,110428848767940912,9.956640e+16,351030,50.0,150.0,1000.0,150.0,1000.0,150.0
16775,1710690,LPL_B:329939,100,861370,100,741592,200,2023-07-16T12:12:44.476Z,9.956640e+16,110428848767941009,9.956640e+16,725032,2300.0,150.0,100.0,200.0,100.0,200.0
16776,1961936,LPL_B:329985,200,883323,100,399185,100,2023-07-17T13:16:45.697Z,9.956640e+16,110428848767875100,9.956640e+16,547021,1250.0,200.0,400.0,800.0,400.0,800.0
16777,1700113,LPL_B:330027,100,885600,100,294277,100,2023-07-17T09:14:18.420Z,9.876799e+16,110428848767940944,9.956640e+16,746019,1350.0,300.0,400.0,500.0,400.0,500.0
